In [1]:
# CELL 1: Setup
%matplotlib inline
import os
import warnings

os.chdir(r"C:\Users\Tugbars\Desktop\Particle-Filter-2D\python")
os.environ['OMP_NUM_THREADS'] = '16'
os.environ['MKL_NUM_THREADS'] = '16'

for p in [r"C:\Program Files (x86)\Intel\oneAPI\mkl\latest\bin",
          r"C:\Program Files (x86)\Intel\oneAPI\compiler\latest\bin"]:
    if os.path.exists(p):
        os.add_dll_directory(p)

warnings.filterwarnings('ignore', message='.*subnormal.*')
warnings.filterwarnings('ignore', message='.*auto_adjust.*')
print("Setup OK")

Setup OK


In [2]:
# CELL 2: Imports
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # Non-interactive backend
import matplotlib.pyplot as plt
import yfinance as yf
import time
from pf2d import create_default_filter
from IPython.display import Image, display

print("Imports OK")

Imports OK


In [3]:
# CELL 3: Fetch data
TICKER = "SPY"
df = yf.download(TICKER, period="1y", progress=False)
prices = df['Close'].values.flatten()
dates = df.index

print(f"{TICKER}: {len(prices)} days, ${prices[0]:.2f} → ${prices[-1]:.2f}")

SPY: 250 days, $599.37 → $684.45


In [ ]:
# CELL 4: Run filter
pf = create_default_filter(n_particles=4000)
pf.set_observation_variance(1.0)
pf.initialize(price0=prices[0], log_vol0=np.log(0.01))

start = time.perf_counter()
price_est, vol_est, ess = pf.run(prices)
elapsed = time.perf_counter() - start

print(f"Done: {len(prices)} ticks in {elapsed*1000:.1f} ms ({elapsed/len(prices)*1e6:.1f} μs/tick)")
print(f"Mean ESS: {np.mean(ess):.0f} ({np.mean(ess)/4000*100:.1f}%)")

In [1]:
# CELL 5: Compute derived quantities
# Velocity (trend)
velocity = np.diff(price_est, prepend=price_est[0])
alpha = 0.2
velocity_smooth = pd.Series(velocity).ewm(alpha=alpha).mean().values

# Annualized volatility
vol_ann = vol_est * np.sqrt(252) * 100

# Realized vol for comparison
returns = np.diff(prices, prepend=prices[0]) / prices
realized_vol = pd.Series(returns).rolling(20).std() * np.sqrt(252) * 100

# NIS (model health)
innovation = prices - price_est
nis = innovation**2 / (vol_est * prices)**2

print("Derived quantities OK")

NameError: name 'np' is not defined

In [ ]:
# CELL 6: 4-panel plot (like your UKF notebook)
fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)

# 1. Price vs Filtered
axes[0].plot(dates, prices, 'b-', lw=1, alpha=0.7, label='Raw Price')
axes[0].plot(dates, price_est, 'r-', lw=1.5, label='Filtered (Particle)')
axes[0].set_ylabel('Price ($)')
axes[0].set_title(f'{TICKER} - Price vs Filtered Level')
axes[0].legend(loc='upper left')
axes[0].grid(True, alpha=0.3)

# 2. Velocity (Trend)
bullish = np.where(velocity_smooth > 0, velocity_smooth, 0)
bearish = np.where(velocity_smooth < 0, velocity_smooth, 0)
axes[1].fill_between(dates, 0, bullish, color='green', alpha=0.7, label='Bullish')
axes[1].fill_between(dates, 0, bearish, color='red', alpha=0.7, label='Bearish')
axes[1].axhline(0, color='black', lw=0.5)
axes[1].set_ylabel('Velocity ($/day)')
axes[1].set_title('Filtered Velocity (Trend Strength)')
axes[1].legend(loc='upper left')
axes[1].grid(True, alpha=0.3)

# 3. Volatility
axes[2].plot(dates, vol_ann, 'purple', lw=1, label='Particle Filter')
axes[2].plot(dates, realized_vol, 'orange', lw=1, alpha=0.7, label='Realized (20d)')
axes[2].axhline(20, color='gray', ls='--', alpha=0.5, label='20%')
axes[2].set_ylabel('Vol (% ann)')
axes[2].set_title('Volatility Estimate')
axes[2].legend(loc='upper right')
axes[2].grid(True, alpha=0.3)
axes[2].set_ylim(0, 60)

# 4. NIS (Model Health)
axes[3].semilogy(dates, np.clip(nis, 1e-3, 100), 'orange', lw=0.8)
axes[3].axhline(1, color='green', ls='--', label='Expected=1')
axes[3].axhline(5, color='red', ls='--', label='Warning')
axes[3].set_ylabel('NIS')
axes[3].set_xlabel('Date')
axes[3].set_title('Normalized Innovation Squared (Model Health)')
axes[3].legend(loc='upper right')
axes[3].grid(True, alpha=0.3)
axes[3].set_ylim(1e-2, 100)

plt.tight_layout()
plt.savefig('pf2d_analysis.png', dpi=150, bbox_inches='tight')
plt.close(fig)

# Display in notebook
display(Image(filename='pf2d_analysis.png'))

In [ ]:
# CELL 7: Statistics
print("="*50)
print("PARTICLE FILTER SUMMARY")
print("="*50)
print(f"\nData: {TICKER}, {len(prices)} days")
print(f"Performance: {elapsed/len(prices)*1e6:.1f} μs/tick")
print(f"\nPrice RMSE: ${np.sqrt(np.mean(innovation**2)):.4f}")
print(f"Mean Vol (ann): {np.nanmean(vol_ann):.1f}%")
print(f"Mean ESS: {np.mean(ess):.0f} ({np.mean(ess)/4000*100:.1f}%)")
print(f"Bullish days: {np.sum(velocity_smooth > 0)} ({np.mean(velocity_smooth > 0)*100:.1f}%)")
print(f"Mean NIS: {np.nanmean(nis):.2f}")